In [20]:
# Python script to scarp WHO for SARS data and clean
# By Chris Chiang
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import datetime as dt
from splinter import Browser
import time
import re


In [3]:
def start_browser():
    # browser init for webscraping
    executable_path = {'executable_path': 'chromedriver.exe'}
    browser = Browser('chrome', **executable_path, headless=False)
    return browser

In [123]:
# setting date parameters
start_date = dt.date(2003, 3,17)
format_change_date1 =dt.date(2003,4,10)
format_change_date2 =dt.date(2003,4,24)
end_date = dt.date(2003, 7,12)
cur_date = start_date
# Setting browers params
output_df=pd.DataFrame({'Date':'', 'Country':'','Cases':'','Deaths':''},index=[])
home ='https://www.who.int/csr/sars/country/en/'
browser=start_browser()
browser.visit(home)
# loop though WHO links
while cur_date!= end_date:
    # search string formatting
    date_string = cur_date.strftime("%d %B %Y")
    if date_string[0]=='0':
        date_string=date_string[1:]
    # Catch some missing dates
    try:
        browser.click_link_by_partial_text(date_string)
        html = bs(browser.html, 'html.parser')
        tab_tag=html.find('table')
        # read html table into df
        temp_df = pd.read_html(str(tab_tag))[0]
        browser.back()
        # Deal with some region name formatting
        if cur_date == dt.date(2003,6,9):
            temp_df = temp_df.iloc[:-1]
        for index, row in temp_df.iterrows():
            if row[0].find('Hong Kong') != -1:
                temp_df.iloc[index,0]="Hong Kong"
            elif row[0].find('Taiwan') != -1:
                temp_df.iloc[index,0]="Taiwan"
            elif row[0].find('Guangdong') != -1:
                temp_df.iloc[index,0]="China"
            elif row[0].find('Macao') != -1:
                temp_df.iloc[index,0]="China"
            elif row[0].find('China +') != -1:
                temp_df.iloc[index,0]="China"
        # table format 1
        if cur_date < format_change_date1:
            temp_df=temp_df.iloc[1:,:3]
            temp_df.rename(columns={0:'Country',1:'Cases',2:'Deaths'},inplace=True)
            temp_df['Date']=cur_date
            temp_df=temp_df[['Date','Country','Cases','Deaths']]
            output_df=output_df.append(temp_df)
        # table format 2
        elif cur_date < format_change_date2:
            temp_df=temp_df.iloc[2:,:4]
            temp_df.rename(columns={0:'Country',1:'Cases',3:'Deaths'},inplace=True)
            temp_df['Date']=cur_date
            temp_df=temp_df[['Date','Country','Cases','Deaths']]
            output_df=output_df.append(temp_df)
        # format 3
        else:
            append_df=pd.DataFrame()
            append_df['Country']=temp_df.iloc[:,0]
            append_df['Cases']=temp_df.iloc[:,1]
            append_df['Deaths']=temp_df.iloc[:,3]
            append_df['Date']=cur_date
            append_df=append_df[['Date','Country','Cases','Deaths']]
            output_df=output_df.append(append_df)
        print(cur_date)
        # Increment day
        cur_date += dt.timedelta(days=1)
    except:
        print('no data for '+date_string)
        cur_date += dt.timedelta(days=1)
browser.quit()
output_df

2003-03-17
2003-03-18
2003-03-19
2003-03-20
2003-03-21
2003-03-22
no data for 23 March 2003
2003-03-24
2003-03-25
2003-03-26
2003-03-27
2003-03-28
2003-03-29
no data for 30 March 2003
2003-03-31
2003-04-01
2003-04-02
2003-04-03
2003-04-04
2003-04-05
2003-04-06
2003-04-07
2003-04-08
2003-04-09
2003-04-10
2003-04-11
2003-04-12
no data for 13 April 2003
2003-04-14
2003-04-15
2003-04-16
2003-04-17
2003-04-18
2003-04-19
no data for 20 April 2003
2003-04-21
2003-04-22
2003-04-23
2003-04-24
2003-04-25
2003-04-26
no data for 27 April 2003
2003-04-28
2003-04-29
2003-04-30
2003-05-01
2003-05-02
2003-05-03
2003-05-04
2003-05-05
2003-05-06
2003-05-07
2003-05-08
2003-05-09
2003-05-10
no data for 11 May 2003
2003-05-12
2003-05-13
2003-05-14
2003-05-15
2003-05-16
2003-05-17
no data for 18 May 2003
2003-05-19
2003-05-20
2003-05-21
2003-05-22
2003-05-23
2003-05-24
no data for 25 May 2003
2003-05-26
2003-05-27
2003-05-28
2003-05-29
2003-05-30
2003-05-31
2003-06-01
2003-06-02
2003-06-03
2003-06-04
2003-0

,Date,Country,Cases,Deaths
1,2003-03-17,Germany,1,0
2,2003-03-17,Canada,8,2
3,2003-03-17,Singapore,20,0
4,2003-03-17,Hong Kong,95,1**
5,2003-03-17,Switzerland,2,0
...,...,...,...,...
28,2003-07-11,Thailand,9,2
29,2003-07-11,United Kingdom,4,0
30,2003-07-11,United States^7,75,0
31,2003-07-11,Viet Nam,63,5


In [231]:
# Set new index 
import numpy as np
output_df=output_df.set_index(np.arange(0,2852))

In [232]:
# cleaning and formating output
# Using regex
# remove a few wrongly inserted rows
output_df_cleaned = output_df[output_df['Country']!='Country']
# Remove special characters in country
alphabet_only = re.compile('[^a-zA-Z ]')
output_df_cleaned['Country']=output_df_cleaned['Country'].apply(lambda x:alphabet_only.sub('',x))
# Remove special characters in Cases and deaths
output_df_cleaned['Cases']=output_df_cleaned['Cases'].fillna('0').apply(lambda x:int(re.sub('[^0-9]','',str(x))))
output_df_cleaned['Deaths']=output_df_cleaned['Deaths'].fillna('0').apply(lambda x:int(re.sub('[^0-9]','',str(x))))



In [233]:
output_df_cleaned.dtypes

Date       object
Country    object
Cases       int64
Deaths      int64
dtype: object

In [234]:
output_df_cleaned['Country'].unique()

array(['Germany', 'Canada', 'Singapore', 'Hong Kong', 'Switzerland',
       'Thailand', 'Viet Nam', 'Total', 'China', 'Taiwan', 'Slovenia',
       'United Kingdom', 'Spain', 'United States', 'Italy',
       'Republic of Ireland', 'France', 'Romania', 'Australia', 'Belgium',
       'Brazil', 'Malaysia', 'Indonesia', 'Japan', 'Kuwait',
       'Philippines', 'South Africa', 'Sweden', 'Viet Nam ', 'India',
       'Mongolia', 'Bulgaria', 'Republic of Korea', 'Poland',
       'New Zealand', 'Colombia', 'Finland', 'Russian Federation'],
      dtype=object)

In [235]:
output_df_cleaned['Deaths'].unique()

array([   0,    2,    1,    4,    5,    9,    6,   10,    7,   11,    3,
         17,   31,   49,   34,   53,   54,   13,   58,   16,   62,   46,
         78,   79,   84,   20,   89,   14,   36,   23,   98,   25,    8,
        103,   27,  106,   55,   30,  111,   32,  116,   35,  119,   64,
         47,   12,  144,   56,  154,   61,  159,   65,   15,  165,   81,
        182,   86,   94,  217,   92,   99,  229,  105,  251,  110,  109,
         19,  263,  115,  274,   18,  122,  121,   21,  293,  131,  138,
        321,  148,  150,   24,  353,  157,  372,  170,  162,  391,   22,
        181,  417,  190,  179,  435,  267,  227,   28,  587,  206,  187,
         26,  461,  214,  193,  478,  219,  204,  495,  224,  208,  506,
        230,  210,  514,  235,  212,  526,  252,  218,  552,  262,  225,
        573,  271,  234,  598,  275,  238,  611,  282,  243,  623,  289,
         40,  643,  294,  253,   52,  662,  296,  255,  666,  300,  258,
         60,   29,  682,  303,  260,  689,  308,  6

In [236]:
output_df_cleaned['Cases'].unique()


array([    1,     8,    20,    95,     2,    40,   167,     0,    23,
         123,     3,    57,   219,   150,    31,    11,    56,   264,
           9,   173,    34,     7,     4,    62,   306,   203,    39,
           6,    13,   350,   222,    44,    22,    63,   386,   260,
          65,     5,    37,    58,   456,   286,    69,   487,    19,
         792,   316,    74,  1323,    28,   806,   367,    78,    45,
        1408,    29,   425,    10,    86,    51,  1485,   470,    89,
          59,  1550,   530,    91,  1622,    53,   685,    92,  1804,
        1190,   708,    72,  2223,   734,    14,    98,    85,  2270,
        1220,   761,    15,   100,  2353,   800,    17,   101,   115,
        2416,   103,  1432,  1268,    27,   162,   193,  3293,    90,
         883,    21,   106,   141,  2601,  1279,   928,   113,   148,
        2671,    94,  1280,   970,   118,   149,  2722,    97,  1290,
         998,   126,   154,  2781,  1309,  1059,   133,   166,  2890,
        1108,   147,

In [237]:
# Save to csv
output_df_cleaned.to_csv('./SARS_data.csv',index=False)